In [1]:
puts `ls -l raw-data`


total 484
-rw-rw-r-- 1 osboxes osboxes  34777 Aug 12 17:45 Demokritos-KG-information.xlsx
-rw-rw-r-- 1 osboxes osboxes 207331 Aug 12 17:45 Disease-Therapeutic_Area.tsv
-rw-rw-r-- 1 osboxes osboxes  36869 Aug 12 17:45 Drug-Diseasetriples.tsv
-rw-rw-r-- 1 osboxes osboxes 111643 Aug 12 17:45 Drug-Drug_type.tsv
-rw-rw-r-- 1 osboxes osboxes  32600 Aug 12 17:45 Drug-Genetriples.tsv
-rw-rw-r-- 1 osboxes osboxes  51800 Aug 12 17:45 Gene-Diseasetriples.tsv
-rw-rw-r-- 1 osboxes osboxes   7965 Aug 12 17:45 Gene-Pathwaytriples.tsv


In [18]:
puts `head -2 "raw-data/Gene-Diseasetriples.tsv"`
puts `head -2 "raw-data/Gene-Pathwaytriples.tsv"`
puts `head -2 "raw-data/Drug-Genetriples.tsv"`


﻿Gene	Gene_id	Disease	Disease_id
WASF1 gene	C1421479	Hereditary Diseases	C0019247
﻿Gene	Gene_id	Pathway	Pathway_id
ATXN3 gene	C1538301	macromolecule modification	C2752381
﻿Drug	Drug_id	Gene	Gene_id
Dopamine Hydrochloride	C0282151	UCP1 gene	C1421313


In [26]:
cuisall = {}
["./raw-data/Drug-Genetriples.tsv","raw-data/Gene-Pathwaytriples.tsv","raw-data/Gene-Diseasetriples.tsv"].each do |fn|
  CSV.foreach(fn, headers: true, col_sep: "\t") do |row|
      cuisall[row["Gene_id"]] = 1  # this is a UMLS CUI
  end
end
cuis = cuisall.keys
puts cuis.length

854


# Column 2 = UMLS CUI

UMLS requires an API key, that has onerous reporting requirements.  Skip it and use mygene.info to find the mapping

need:  source,label,geneid,protein,recommended_full,taxon

## NOTE:  C1418941  is a prion protein that breaks the parser... why?

In [30]:
# puts `curl -H "Accept: application/json" "https://mygene.info/v3/query?q=C1538301&fields=symbol,ensembl.gene,uniprot"`
require "rest-client"
require 'json'

resp =  RestClient.get("https://mygene.info/v3/query?q=C1418941&fields=symbol,name,entrezgene,uniprot,taxid")
puts resp

  data = JSON.parse(resp)
  hit = data.dig('hits', 0)
  genesymbol = hit&.dig('symbol')
  geneid = hit&.dig('entrezgene')
  recommended_full = hit&.dig('name')
  taxon = hit&.dig('taxid')
  protein = hit&.dig('uniprot','Swiss-Prot')

puts protein

{"took":6,"total":1,"max_score":1.7734557,"hits":[{"_id":"5621","_score":1.7734557,"entrezgene":"5621","name":"prion protein (Kanno blood group)","symbol":"PRNP","taxid":9606,"uniprot":{"Swiss-Prot":["P04156","F7VJQ1"],"TrEMBL":"Q53YK7"}}]}
P04156
F7VJQ1


In [28]:
require "rest-client"
require "json"

def map_cui_to_geneinfo(cui)
  response = RestClient.get("https://mygene.info/v3/query?q=#{cui}&fields=symbol,name,entrezgene,uniprot,taxid")

  data = JSON.parse(response)
  hit = data.dig('hits', 0)
  genesymbol = hit&.dig('symbol')
  geneid = hit&.dig('entrezgene')
  recommended_full = hit&.dig('name')
  taxon = hit&.dig('taxid')
  protein = hit&.dig('uniprot','Swiss-Prot')
  protein = protein.first if protein.is_a? Array  # the prion!  darn it!
  
  if geneid && protein
    { source: cui, geneid: geneid, genesymbol: genesymbol, recommended_full: recommended_full, taxon: taxon, protein: protein }
  else
    warn "No data found for #{cui}"
    return false
  end
rescue StandardError => e
   warn "Error for #{cui}: #{e.inspect}"
  return false
end


:map_cui_to_geneinfo

In [29]:
require 'csv'
puts `pwd`

f = File.open('./mappings/gene-mappings.map', 'w')
e = File.open('./mappings/gene-errors.txt', 'w')
f.sync = true # Ensure immediate writes
e.sync = true # Ensure immediate writes
f.write CSV.generate_line(["source","label","geneid","protein","recommended_full","taxon"])

cuis.each do |cui|

  result = map_cui_to_geneinfo(cui) # {:cui=>"C1538301", :gene_name=>"ATXN3", :ensembl=>"ENSG00000066427"}

  if result == false
    e.write "error getting #{cui}\n"
    next
  end

  warn result[:geneid]
#   source: cui, geneid: geneid, genesymbol: genesymbol, recommended_full: recommended_full, taxon: taxon, protein: protein }
  f.write CSV.generate_line([
    cui, 
    result[:genesymbol], 
    "http://purl.uniprot.org/geneid/#{result[:geneid]}",
    "http://purl.uniprot.org/uniprot/#{result[:protein]}",
    result[:recommended_full],
    "http://purl.uniprot.org/taxonomy/#{result[:taxon]}"])
    
end
f.close
e.close
puts "DONE"

/home/osboxes/SIMPATHIC_Aug12/SKG_Mapping/demokritos


7350
5465
4659
No data found for C3538756
No data found for C3813713
5199
55364
1812
4287
6571
11273
No data found for C1883559
7166
55768
No data found for C2239937
94233
2890
340061
2903
2904
6532
1392
84254
2932
331
1565
4025
1644
60482
1825
27235
2110
2965
1760
23543
5251
5194
4129
274
23788
23787
64849
2030
4154
8170
10891
10658
1180
No data found for C3539645
255324
1128
23241
1052
6608
5580
1268
4654
1536
4656
7157
29978
6536
1269
51497
2617
4644
706
746
1603
5707
100128731
9197
1650
5542
84961
23034
23411
6850
222662
5805
133522
577
5663
3196
581
8878
No data found for C1333402
No data found for C3812695
23774
6717
5828
6648
No data found for C1332838
54205
No data found for C0206522
2000
4014
83650
No data found for C3539639
84061
2639
No data found for C3658343
No data found for C3539530
No data found for C3715044
29926
No data found for C0002085
5373
1489
4728
2673
78996
3061
11343
5552
6558
4724
6834
9788
No data found for C3541424
4515
5160
89885
285489
51643
5179
22794
30

DONE
